**[Natural Language Processing Home Page](https://www.kaggle.com/learn/natural-language-processing)**

---


# Basic Text Processing with Spacy
    
You're a consultant for [DelFalco's Italian Restaurant](https://defalcosdeli.com/index.html).
The owner asked you to identify whether there are any foods on their menu that diners find disappointing. 

<img src="https://i.imgur.com/8DZunAQ.jpg" alt="Meatball Sub" width="250"/>

Before getting started, run the following cell to set up code checking.

In [2]:
import pandas as pd

# Set up code checking
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
from learntools.core import binder
binder.bind(globals())
from learntools.nlp.ex1 import *
print('Setup Complete')

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-oagfhztc
ERROR: Command errored out with exit status 128: git clone -q https://github.com/Kaggle/learntools.git /tmp/pip-req-build-oagfhztc Check the logs for full command output.
Setup Complete


The business owner suggested you use diner reviews from the Yelp website to determine which dishes people liked and disliked. You pulled the data from Yelp. Before you get to analysis, run the code cell below for a quick look at the data you have to work with.

In [3]:
# Load in the data from JSON file
data = pd.read_json('../input/nlp-course/restaurant.json')
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


In [4]:
data.text.iloc[14]

"The Il Purista sandwich has become a staple of my life. Mozzarella, basil, prosciutto, roasted red peppers and balsamic vinaigrette blend into a front runner for the best sandwich in the valley. Goes great with sparkling water or a beer. \n\nDeFalco's also has other Italian fare such as a delicious meatball sub and classic pastas."

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321 entries, 109 to 644757
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   review_id    1321 non-null   object        
 1   user_id      1321 non-null   object        
 2   business_id  1321 non-null   object        
 3   stars        1321 non-null   int64         
 4   useful       1321 non-null   int64         
 5   funny        1321 non-null   int64         
 6   cool         1321 non-null   int64         
 7   text         1321 non-null   object        
 8   date         1321 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 103.2+ KB


The owner also gave you this list of menu items and common alternate spellings.

In [6]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

# Step 1: Plan Your Analysis

Given the data from Yelp and the list of menu items, do you have any ideas for how you could find which menu items have disappointed diners?

Think about your answer. Then run the cell below to see one approach.

In [7]:
# Check your answer (Run this code cell to receive credit!)
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> You could group reviews by what menu items they mention, and then calculate the average rating
    for reviews that mentioned each item. You can tell which foods are mentioned in reviews with low scores,
    so the restaurant can fix the recipe or remove those foods from the menu.

# Step 2: Find items in one review

You'll pursue this plan of calculating average scores of the reviews mentioning each menu item.

As a first step, you'll write code to extract the foods mentioned in a single review.

Since menu items are multiple tokens long, you'll use `PhraseMatcher` which can match series of tokens.

Fill in the `____` values below to get a list of items matching a single menu item.

In [8]:
menu

['Cheese Steak',
 'Cheesesteak',
 'Steak and Cheese',
 'Italian Combo',
 'Tiramisu',
 'Cannoli',
 'Chicken Salad',
 'Chicken Spinach Salad',
 'Meatball',
 'Pizza',
 'Pizzas',
 'Spaghetti',
 'Bruchetta',
 'Eggplant',
 'Italian Beef',
 'Purista',
 'Pasta',
 'Calzones',
 'Calzone',
 'Italian Sausage',
 'Chicken Cutlet',
 'Chicken Parm',
 'Chicken Parmesan',
 'Gnocchi',
 'Chicken Pesto',
 'Turkey Sandwich',
 'Turkey Breast',
 'Ziti',
 'Portobello',
 'Reuben',
 'Mozzarella Caprese',
 'Corned Beef',
 'Garlic Bread',
 'Pastrami',
 'Roast Beef',
 'Tuna Salad',
 'Lasagna',
 'Artichoke Salad',
 'Fettuccini Alfredo',
 'Chicken Parmigiana',
 'Grilled Veggie',
 'Grilled Veggies',
 'Grilled Vegetable',
 'Mac and Cheese',
 'Macaroni',
 'Prosciutto',
 'Salami']

In [9]:
import spacy
from spacy.matcher import PhraseMatcher

index_of_review_to_test_on = 14
text_to_test_on = data.text.iloc[index_of_review_to_test_on]

# Load the SpaCy model
nlp = spacy.blank('en')

# Create the tokenized version of text_to_test_on
review_doc = nlp(text_to_test_on)

# Create the PhraseMatcher object. The tokenizer is the first argument. Use attr = 'LOWER' to make consistent capitalization
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

# Create a list of tokens for each item in the menu
menu_tokens_list = [nlp(item) for item in menu]

# Add the item patterns to the matcher. 
# Look at https://spacy.io/api/phrasematcher#add in the docs for help with this step
# Then uncomment the lines below 


matcher.add("MENU",            # Just a name for the set of rules we're matching to
           menu_tokens_list
          )

# Find matches in the review_doc
matches = matcher(review_doc)

# Uncomment to check your work
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [10]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

After implementing the above cell, uncomment the following cell to print the matches.

In [11]:
for match in matches:
   print(f"Token number {match[1]}: {review_doc[match[1]:match[2]]}")

Token number 2: Purista
Token number 16: prosciutto
Token number 58: meatball


# Step 3: Matching on the whole dataset

Now run this matcher over the whole dataset and collect ratings for each menu item. Each review has a rating, `review.stars`. For each item that appears in the review text (`review.text`), append the review's rating to a list of ratings for that item. The lists are kept in a dictionary `item_ratings`.

To get the matched phrases, you can reference the `PhraseMatcher` documentation for the structure of each match object:

>A list of `(match_id, start, end)` tuples, describing the matches. A match tuple describes a span `doc[start:end]`. The `match_id` is the ID of the added match pattern.

In [28]:
from collections import defaultdict

# item_ratings is a dictionary of lists. If a key doesn't exist in item_ratings,
# the key is added with an empty list as the value.
item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review.text)
    # Using the matcher from the previous exercise
    matches = matcher(doc)
    # Create a set of the items found in the review text
    found_items = set([doc[x[1]:x[2]].lower_ for x in matches])
    # Update item_ratings with rating for each item in found_items
    # Transform the item strings to lowercase to make it case insensitive
    for found in found_items:
        item_ratings[found].append(review.stars)

q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [25]:
# Lines below will give you a hint or solution code
q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> For each review, use the `nlp` model to convert the text to a document. Then use the matcher from exercise 1 to extract the item matches from the review text. The matches you get from the matcher are tuples (match_id, start, end), so you can do doc[start:end] to get the text phrase for that match. To get all of the unique items in the review, create a list of all the matched phrases, and convert that into a set. Finally for each of those items, append the review's rating to item_ratings. Make sure to add the item string in lowercase. 

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

from collections import defaultdict

item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review.text)
    matches = matcher(doc)

    found_items = set([doc[match[1]:match[2]].lower_ for match in matches])

    for item in found_items:
        item_ratings[item].append(review.stars)

```

# Step 4: What's the worst reviewed item?

Using these item ratings, find the menu item with the worst average rating.

In [33]:
# Calculate the mean ratings for each menu item as a dictionary
mean_ratings = {key: sum(value) / len(value) for key,value in item_ratings.items()}
print(mean_ratings)
# Find the worst item, and write it as a string in worst_text. This can be multiple lines of code if you want.
worst_item = sorted(mean_ratings.items(), key=lambda x: x[1])[0][0]


q_4.check()

{'chicken parmigiana': 4.470588235294118, 'eggplant': 4.159420289855072, 'steak and cheese': 4.888888888888889, 'pizza': 4.339622641509434, 'meatball': 4.1796875, 'pasta': 4.407766990291262, 'cannoli': 4.388888888888889, 'purista': 4.666666666666667, 'prosciutto': 4.68, 'cheese steak': 4.447368421052632, 'cheesesteak': 4.484536082474227, 'calzone': 4.444444444444445, 'italian combo': 4.0476190476190474, 'chicken spinach salad': 4.5, 'tiramisu': 4.238095238095238, 'italian beef': 3.92, 'salami': 4.25, 'chicken parm': 4.22, 'tuna salad': 4.0, 'chicken pesto': 4.555555555555555, 'ziti': 4.380952380952381, 'turkey sandwich': 3.8, 'chicken cutlet': 3.4, 'lasagna': 4.4576271186440675, 'artichoke salad': 5.0, 'fettuccini alfredo': 5.0, 'pizzas': 4.375, 'turkey breast': 5.0, 'calzones': 4.542857142857143, 'mac and cheese': 4.454545454545454, 'grilled veggie': 4.5, 'garlic bread': 4.128205128205129, 'spaghetti': 3.888888888888889, 'italian sausage': 4.30188679245283, 'portobello': 4.5, 'pastram

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [32]:
# Lines below will give you a hint or solution code
q_4.hint()
q_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Loop through each item in item_ratings and calculate the mean, the sum of the ratings divided by the number of ratings. This is easiest using a dictionary comprehension. Then use the `sorted` function to sort the dictionary keys based on the dictionary values.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

# There are many ways to do this. Here is one.
mean_ratings = {item: sum(ratings)/len(ratings) for item, ratings in item_ratings.items()}
worst_item = sorted(mean_ratings, key=mean_ratings.get)[0]

```

In [34]:
# After implementing the above cell, uncomment and run this to print 
# out the worst item, along with its average rating. 

print(worst_item)
print(mean_ratings[worst_item])

chicken cutlet
3.4


# Step 5: Are counts important here?

Similar to the mean ratings, you can calculate the number of reviews for each item.

In [35]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}

item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}{counts[item]:>5}")

                    pizza  265
                    pasta  206
                 meatball  128
              cheesesteak   97
             cheese steak   76
                  cannoli   72
                  calzone   72
                 eggplant   69
                  purista   63
                  lasagna   59
          italian sausage   53
               prosciutto   50
             chicken parm   50
             garlic bread   39
                  gnocchi   37
                spaghetti   36
                 calzones   35
                   pizzas   32
                   salami   28
            chicken pesto   27
             italian beef   25
            italian combo   21
                 tiramisu   21
                     ziti   21
         chicken parmesan   19
       chicken parmigiana   17
               portobello   14
           mac and cheese   11
           chicken cutlet   10
         steak and cheese    9
                 pastrami    9
               roast beef    7
       f

![](http://)Here is code to print the 10 best and 10 worst rated items. Look at the results, and decide whether you think it's important to consider the number of reviews when interpreting scores of which items are best and worst.

In [ ]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

Run the following line after you've decided your answer.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_5.solution()

# Keep Going

Now that you are ready to combine your NLP skills with your ML skills, **[see how it's done](https://www.kaggle.com/matleonard/text-classification)**.

---
**[Natural Language Processing Home Page](https://www.kaggle.com/learn/natural-language-processing)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161466) to chat with other Learners.*